# Clustering the Neighbourhoods of London


### 1. Introduction

London is a quite the popular tourist and vacation destination for people all around the world. It is diverse and multicultural and offer a wide variety of experiences that is widely sought after. We try to group the neighbourhoods of London and draw insights to what they look like now.

### 2. Business Problem
The aim is to help tourists choose their destinations depending on the experiences that the neighbourhoods have to offer and what they would want to have. This also helps people make decisions if they are thinking about migrating to London or even if they want to relocate neighbourhoods within the city. Our findings will help stakeholders make informed decisions and address any concerns they have including the different kinds of cuisines, provision stores and what the city has to offer.

### 3. Data
We require geographical location data for both London and Paris. Postal codes in each city serve as a starting point. Using Postal codes we use can find out the neighborhoods, boroughs, venues and their most popular venue categories.


#### London

We scrape our data from https://en.wikipedia.org/wiki/List_of_areas_of_London
on:
* borough : Name of Neighbourhood
* town : Name of borough
* post_code : Postal codes for London.

and we use ArcGIS API:

* ArcGIS API

ArcGIS Online enables you to connect people, locations, and data using interactive maps. Work with smart, data-driven styles and intuitive analysis tools that deliver location intelligence. Share your insights with the world or specific groups.

More specifically, we use ArcGIS to get the geo locations of the neighbourhoods of London. The following columns are added to our initial dataset which prepares our data.

latitude : Latitude for Neighbourhood
longitude : Longitude for Neighbourhood

##### Foursquare API Data

We will need data about different venues in different neighbourhoods of that specific borough. In order to gain that information we will use "Foursquare" locational information. Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

After finding the list of neighbourhoods, we then connect to the Foursquare API to gather information about venues inside each and every neighbourhood. For each neighbourhood, we have chosen the radius to be 500 meters.

The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

* Neighbourhood : Name of the Neighbourhood
* Neighbourhood Latitude : Latitude of the Neighbourhood
* Neighbourhood Longitude : Longitude of the Neighbourhood
* Venue : Name of the Venue
* Venue Latitude : Latitude of Venue
* Venue Longitude : Longitude of Venue
* Venue Category : Category of Venue

### 4. Methodology

In [2]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# import k-means for the clustering stage
from sklearn.cluster import KMeans

The approach taken here is to explore each of the cities individually, plot the map to show the neighbourhoods being considered and then build our model by clustering all of the similar neighbourhoods together and finally plot the new map with the clustered neighbourhoods. We draw insights and then compare and discuss our findings.

##### Neighbourhoods of London
We begin to start collecting and refining the data needed for the our business solution to work.

In [3]:
url_london = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_london_url = requests.get(url_london)
wiki_london_url

<Response [200]>

In [7]:
wiki_london_data = pd.read_html(wiki_london_url.text)
wiki_london_data

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                       Download coordinates as: KML,
             Location                     London borough       Post town  \
 0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
 1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
 2          Addington                         Croydon[8]         CROYDON   
 3         Addiscombe                         Croydon[8]         CROYDON   
 4        Albany Park                             Bexley  BEXLEY, SIDCUP   
 ..               ...                                ...             ...   
 526         Woolwich                          Greenwich          LONDON   
 527   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
 528  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
 529          Yeading                         Hillingdon           HAYES   
 

In [8]:
wiki_london_data = wiki_london_data[1]
wiki_london_data

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


Data Preprocessing
we remove the spaces in the column titles and then we add _ between words.

In [9]:
wiki_london_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
wiki_london_data

,Location,London borough,Post_town,Postcode district,Dial code,OS_grid_ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


#### Feature Selection
We need only the boroughs, Postal codes, Post town for further steps. We can drop the locations, dial codes and OS grid.

In [10]:
df1 = wiki_london_data.drop( [ wiki_london_data.columns[0], wiki_london_data.columns[4], wiki_london_data.columns[5] ], axis=1)
df1.head()

,London borough,Post_town,Postcode district
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [11]:
df1.columns = ['borough','town','post_code']
df1

,borough,town,post_code
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
526,Greenwich,LONDON,SE18
527,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
528,Hammersmith and Fulham,LONDON,W12
529,Hillingdon,HAYES,UB4


Let's remove the Square brackets [ ] and numbers from the borough column

In [12]:
df1['borough'] = df1['borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Croydon,CROYDON,CR0
3,Croydon,CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
526,Greenwich,LONDON,SE18
527,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
528,Hammersmith and Fulham,LONDON,W12
529,Hillingdon,HAYES,UB4


In [13]:
df1.shape

(531, 3)

We currently have 531 records and 3 columns of our data. It's time to perform Feature Engineering

#### Feature Engineering

In [14]:
df1 = df1[df1['town'].str.contains('LONDON')]
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
6,City,LONDON,EC3
7,Westminster,LONDON,WC2
9,Bromley,LONDON,SE20
...,...,...,...
521,Redbridge,LONDON,"IG8, E18"
522,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8
525,Barnet,LONDON,N12
526,Greenwich,LONDON,SE18


In [15]:
df1.shape

(308, 3)

We now have only 308 rows. We can proceed with our further steps. Getting some descriptive statistics

In [16]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308 entries, 0 to 528
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   borough    308 non-null    object
 1   town       308 non-null    object
 2   post_code  308 non-null    object
dtypes: object(3)
memory usage: 9.6+ KB


## Geolocations of the London Neighbourhoods


#### ArcGis API
We need to get the geographical co-ordinates for the neighbourhoods to plot out map. We will use the arcgis package to do so.

Arcgis doesn't have a limitation on the number of API calls made so it fits our use case perfectly.

In [13]:
conda install -c esri arcgis

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - arcgis


The following NEW packages will be INSTALLED:

  arcgis             esri/win-64::arcgis-1.8.2-py38_1275
  blinker            pkgs/main/win-64::blinker-1.4-py38haa95532_0
  lerc               esri/noarch::lerc-2.2-py_0
  ntlm-auth          esri/noarch::ntlm-auth-1.4.0-py_0
  oauthlib           pkgs/main/noarch::oauthlib-3.1.0-py_0
  pyjwt              pkgs/main/win-64::pyjwt-2.1.0-py38haa95532_0
  pyshp              pkgs/main/noarch::pyshp-2.1.3-pyhd3eb1b0_0
  pywin32-security   esri/win-64::pywin32-security-228-py38_3
  requests-kerberos  esri/win-64::requests-kerberos-0.12.0-0
  requests-oauthlib  pkgs/main/noarch::requests-oauthlib-1.3.0-py_0
  requests-toolbelt  pkgs/main/noarch::requests-toolbelt-0.9.1-py_0
  requests_ntlm      esri/noarch::requests_ntlm-1.1.0-py_0
  winkerberos        esri/win-64::winkerberos-0.7.0-py38_0


Preparin

In [17]:
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

Defining London arcgis geocode function to return latitude and longitude

In [18]:
def get_x_y_uk(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, London, England, GBR'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [19]:
c = get_x_y_uk('SE2')
c

'51.499741450000045,0.12406135200006929'

In [20]:
geo_coordinates_uk = df1['post_code']    
geo_coordinates_uk

0           SE2
1        W3, W4
6           EC3
7           WC2
9          SE20
         ...   
521    IG8, E18
522         IG8
525         N12
526        SE18
528         W12
Name: post_code, Length: 308, dtype: object

In [21]:
coordinates_latlng_uk = geo_coordinates_uk.apply(lambda x: get_x_y_uk(x))
coordinates_latlng_uk

0       51.499741450000045,0.12406135200006929
1        51.49776500000007,-0.2558519459999502
6      51.513145000000065,-0.07873298599997725
7       51.51462500000008,-0.11486033199997792
9        51.48249000000004,0.11919361600007505
                        ...                   
521     51.51180000000005,-0.07129021599996577
522    51.507408360000056,-0.12769869299995662
525    51.542635000000075,-0.09858089899995548
526     51.50312900000006,-0.10802518599996347
528    51.515085000000056,-0.24269643599996016
Name: post_code, Length: 308, dtype: object

#### Latitude

In [22]:
lat_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[0])
lat_uk

0      51.499741450000045
1       51.49776500000007
6      51.513145000000065
7       51.51462500000008
9       51.48249000000004
              ...        
521     51.51180000000005
522    51.507408360000056
525    51.542635000000075
526     51.50312900000006
528    51.515085000000056
Name: post_code, Length: 308, dtype: object

#### Longitude

In [23]:
lng_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[1])
lng_uk

0       0.12406135200006929
1       -0.2558519459999502
6      -0.07873298599997725
7      -0.11486033199997792
9       0.11919361600007505
               ...         
521    -0.07129021599996577
522    -0.12769869299995662
525    -0.09858089899995548
526    -0.10802518599996347
528    -0.24269643599996016
Name: post_code, Length: 308, dtype: object

We proceed with Merging our source data with the geographical co-ordinates to make our dataset ready for the next stage

In [24]:
london_merged = pd.concat([df1,lat_uk.astype(float), lng_uk.astype(float)], axis=1)
london_merged.columns= ['borough','town','post_code','latitude','longitude']
london_merged

,borough,town,post_code,latitude,longitude
0,"Bexley, Greenwich",LONDON,SE2,51.499741,0.124061
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.497765,-0.255852
6,City,LONDON,EC3,51.513145,-0.078733
7,Westminster,LONDON,WC2,51.514625,-0.114860
9,Bromley,LONDON,SE20,51.482490,0.119194
...,...,...,...,...,...
521,Redbridge,LONDON,"IG8, E18",51.511800,-0.071290
522,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8,51.507408,-0.127699
525,Barnet,LONDON,N12,51.542635,-0.098581
526,Greenwich,LONDON,SE18,51.503129,-0.108025


In [25]:
london_merged.dtypes

borough       object
town          object
post_code     object
latitude     float64
longitude    float64
dtype: object

#### Co-ordinates for London
Getting the geocode for London to help visualize it on the map

In [26]:
london = geocode(address='London, England, GBR')[0]
london_lng_coords = london['location']['x']
london_lat_coords = london['location']['y']
london_lng_coords

-0.12769869299995662

In [27]:
london_lat_coords

51.507408360000056

### Visualize the Map of London
To help visualize the Map of London and the neighbourhoods in London, we make use of the folium package.

In [28]:
# Creating the map of London
map_London = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)
map_London

# adding markers to map
for latitude, longitude, borough, town in zip(london_merged['latitude'], london_merged['longitude'], london_merged['borough'], london_merged['town']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_London)  
    
map_London

In [29]:
CLIENT_ID = 'ZRGCJRGM40TE5E1DKM5JGSXOHLUZ3MG4OPKQQCFTJBK3H0V4' 
CLIENT_SECRET = 'GB5ERHI2CPJE5HYILZWQACR5YBB5ANNWBXWO444IX05LC3LI'
VERSION = '20180605' # Foursquare API version

In [30]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
venues_in_London = getNearbyVenues(london_merged['borough'], london_merged['latitude'], london_merged['longitude'])

Bexley, Greenwich 
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Barnet
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Islington
Wandsworth
Westminster
Bromley
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
City
Lewisham
Greenwich
Tower Hamlets
Camden
Haringey
Tower Hamlets
Haringey
Barnet
Brent
Lambeth
Lewisham
Tower Hamlets
Kensington and Chelsea, Hammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Tower Hamlets
Newham
Hackney
Islington
Southwark
Lewisham
Brent
Southwark
Ealing
Kensington and Chelsea
Wandsworth
Southwark
Barnet
Newham
Richmond upon Thames


In [32]:
venues_in_London.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Bexley, Greenwich",51.499741,0.124061,Southmere Lake,Lake
1,"Ealing, Hammersmith and Fulham",51.497765,-0.255852,Lara Restaurant,Mediterranean Restaurant
2,"Ealing, Hammersmith and Fulham",51.497765,-0.255852,Hack & Veldt,Coffee Shop
3,"Ealing, Hammersmith and Fulham",51.497765,-0.255852,Good Boy Coffee,Coffee Shop
4,"Ealing, Hammersmith and Fulham",51.497765,-0.255852,Chief Coffee,Coffee Shop


In [33]:
venues_in_London.shape

(12339, 5)

This will definitely make the clustering interesting.



### Grouping by Venue Categories
We need to now see how many Venue Categories are there for further processing

In [34]:
venues_in_London.groupby('Venue Category').max()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Accessories Store,"Camden, Islington",51.517165,-0.126810,James Smith & Sons
Adult Boutique,Hackney,51.529675,-0.083470,Sh! Women's Erotic Emporium
Afghan Restaurant,Enfield,51.535185,-0.100543,Afghan Kitchen
African Restaurant,Westminster,51.586805,-0.065515,Le Chamarel
American Restaurant,Westminster,51.515390,0.029115,When Mac Met Cheese
...,...,...,...,...
Wine Bar,Westminster,51.557670,-0.069523,Vivat Bacchus
Wine Shop,Westminster,51.553177,-0.061776,Wine Pantry
Winery,Lambeth,51.463370,-0.115820,The Wine Parlour



### One Hot Encoding
We need to Encode our venue categories to get a better result for our clustering

In [35]:
London_venue_cat = pd.get_dummies(venues_in_London[['Venue Category']], prefix="", prefix_sep="")
London_venue_cat

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Adding Neighbourhood into the mix.



In [36]:
London_venue_cat['Neighbourhood'] = venues_in_London['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [London_venue_cat.columns[-1]] + list(London_venue_cat.columns[:-1])
London_venue_cat = London_venue_cat[fixed_columns]

London_venue_cat.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Ealing, Hammersmith and Fulham",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Ealing, Hammersmith and Fulham",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Ealing, Hammersmith and Fulham",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Ealing, Hammersmith and Fulham",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Venue categories mean value

We will group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood

In [38]:
London_grouped = London_venue_cat.groupby('Neighbourhood').mean().reset_index()
London_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barnet,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.009901,0.0,0.013861,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,"Barnet, Brent, Camden",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Bexley,0.0,0.0,0.0,0.009709,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.029126,0.0,0.0,0.0,0.0
3,"Bexley, Greenwich",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,"Bexley, Greenwich",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#### Top venue categories
Getting the top venue categories in London

In [41]:
# create a new dataframe for London
neighborhoods_venues_sorted_london = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_london['Neighbourhood'] = London_grouped['Neighbourhood']

for ind in np.arange(London_grouped.shape[0]):
    neighborhoods_venues_sorted_london.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_london.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Pub,Park,Café,Coffee Shop,Bakery,Gastropub,Bus Stop,Cocktail Bar,Indian Restaurant,Train Station
1,"Barnet, Brent, Camden",Pizza Place,Park,Home Service,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
2,Bexley,Hotel,Theater,Plaza,Pub,Monument / Landmark,Cocktail Bar,Wine Bar,Bakery,Lake,English Restaurant
3,"Bexley, Greenwich",Indian Restaurant,Real Estate Office,Furniture / Home Store,Grocery Store,Flea Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
4,"Bexley, Greenwich",Lake,Yoga Studio,Filipino Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


# Model Building

## K Means

Let's cluster the city of london to roughly 5 to make it easier to analyze.

We use the K Means clustering technique to do so.

In [42]:
# set number of clusters
k_num_clusters = 5

London_grouped_clustering = London_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_london = KMeans(n_clusters=k_num_clusters, random_state=0).fit(London_grouped_clustering)
kmeans_london

KMeans(n_clusters=5, random_state=0)

In [43]:
kmeans_london.labels_

array([0, 3, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [44]:
neighborhoods_venues_sorted_london.insert(0, 'Cluster Labels', kmeans_london.labels_ +1)

In [45]:
london_data = london_merged

london_data = london_data.join(neighborhoods_venues_sorted_london.set_index('Neighbourhood'), on='borough')

london_data.head()

,borough,town,post_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bexley, Greenwich",LONDON,SE2,51.499741,0.124061,2,Lake,Yoga Studio,Filipino Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.497765,-0.255852,1,Coffee Shop,Park,Playground,Health Food Store,Fish & Chips Shop,French Restaurant,Fruit & Vegetable Store,Mediterranean Restaurant,Garden Center,Comedy Club
6,City,LONDON,EC3,51.513145,-0.078733,1,Coffee Shop,Hotel,Italian Restaurant,Pub,Gym / Fitness Center,Wine Bar,Cocktail Bar,French Restaurant,Restaurant,Sandwich Place
7,Westminster,LONDON,WC2,51.514625,-0.114860,1,Hotel,Pub,Coffee Shop,Café,French Restaurant,Sandwich Place,Theater,Lounge,Art Gallery,Restaurant
9,Bromley,LONDON,SE20,51.482490,0.119194,1,Bus Station,Campground,Forest,Athletics & Sports,Café,Gym / Fitness Center,Coffee Shop,Convenience Store,Park,Japanese Restaurant


In [46]:
london_data_nonan = london_data.dropna(subset=['Cluster Labels'])


## Visualizing the clustered neighbourhood

Let's plot the clusters

In [47]:
map_clusters_london = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_data_nonan['latitude'], london_data_nonan['longitude'], london_data_nonan['borough'], london_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_london)
        
map_clusters_london


### Examining our Clusters

##### cluster 1

In [48]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 1, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,LONDON,1,Coffee Shop,Park,Playground,Health Food Store,Fish & Chips Shop,French Restaurant,Fruit & Vegetable Store,Mediterranean Restaurant,Garden Center,Comedy Club
6,LONDON,1,Coffee Shop,Hotel,Italian Restaurant,Pub,Gym / Fitness Center,Wine Bar,Cocktail Bar,French Restaurant,Restaurant,Sandwich Place
7,LONDON,1,Hotel,Pub,Coffee Shop,Café,French Restaurant,Sandwich Place,Theater,Lounge,Art Gallery,Restaurant
9,LONDON,1,Bus Station,Campground,Forest,Athletics & Sports,Café,Gym / Fitness Center,Coffee Shop,Convenience Store,Park,Japanese Restaurant
10,LONDON,1,Pub,Café,Coffee Shop,Bus Stop,Cocktail Bar,Bar,Thai Restaurant,Grocery Store,Indian Restaurant,Sandwich Place
...,...,...,...,...,...,...,...,...,...,...,...,...
521,LONDON,1,Hotel,Coffee Shop,Indian Restaurant,Café,Pub,Pizza Place,Gym / Fitness Center,Cocktail Bar,Bar,Middle Eastern Restaurant
522,"LONDON, WOODFORD GREEN",1,Hotel,Coffee Shop,Indian Restaurant,Pub,Café,Cocktail Bar,Theater,Art Gallery,Sandwich Place,Plaza
525,LONDON,1,Pub,Park,Café,Coffee Shop,Bakery,Gastropub,Bus Stop,Cocktail Bar,Indian Restaurant,Train Station
526,LONDON,1,Coffee Shop,Pub,Hotel,Bar,Gym / Fitness Center,Café,Italian Restaurant,Bakery,Grocery Store,Thai Restaurant


##### cluster 2

In [49]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 2, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,LONDON,2,Lake,Yoga Studio,Filipino Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


##### cluster 3

In [50]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 3, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
167,"LONDON, WELLING",3,Indian Restaurant,Real Estate Office,Furniture / Home Store,Grocery Store,Flea Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
457,"LONDON, ERITH",3,Indian Restaurant,Real Estate Office,Furniture / Home Store,Grocery Store,Flea Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant


##### cluster 4

In [51]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 4, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,LONDON,4,Pizza Place,Park,Home Service,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm


##### cluster 5

In [52]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 5, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
270,LONDON,5,Grocery Store,Outdoors & Recreation,Stables,Italian Restaurant,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm


--------------------------------


# Results and Discussion

The neighbourhoods of London are very mulitcultural. There are a lot of different cusines including Indian, Italian, Turkish and Chinese. London seems to take a step further in this direction by having a lot of Restaurants, bars, juice bars, coffee shops, Fish and Chips shop and Breakfast spots. It has a lot of shopping options too with that of the Flea markets, flower shops, fish markets, Fishing stores, clothing stores. The main modes of transport seem to be Buses and trains. For leisure, the neighbourhoods are set up to have lots of parks, golf courses, zoo, gyms and Historic sites.

Overall, the city of London offers a multicultural, diverse and certainly an entertaining experience.

--------------------------------


# Conclusion

The purpose of this project was to explore the city of London, and see how attractive it is to potential tourists and migrants. We explored the city based on its postal codes and then extrapolated the common venues present in each of the neighbourhoods finally concluding with clustering similar neighbourhoods together.

We could see that each of the neighbourhoods in London have a wide variety of experiences to offer which is unique in it's own way. The cultural diversity is quite evident which also gives the feeling of a sense of inclusion.

London seems to offer a vacation stay or a romantic gateaway with a lot of places to explore, beautiful landscapes and a wide variety of culture. 